In [7]:
import torch
from   torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [8]:
import json, os
from pathlib import Path

In [54]:
class SimpleCNN(nn.Module):
    def __init__(self, n_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3)
        self.dropout = nn.Dropout()    
        self.fc1 = nn.Linear(4*26*26,100)
        self.fc2 = nn.Linear(100, n_classes)
            
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = x.view(4,4*26*26)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x


net = SimpleCNN(10)


In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [25]:
config = json.load(open(os.path.expanduser("~/.thesis.conf")))
root = Path(config['datasets']) / Path("MNIST/") 
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
mnist_set = torchvision.datasets.MNIST(str(root),train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(mnist_set, batch_size=4,
                                          shuffle=True, num_workers=2)

dataiter = iter(trainloader)
images, label = dataiter.next()

In [55]:
output = net(Variable(images))